<a href="https://colab.research.google.com/github/danielmendez07/Ponderada-Programacao-16_08_25/blob/main/Ponderada_Programacao_Oficial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto

## Introdução

Este projeto usa o **UCI Bank Marketing – “bank-additional-full.csv”** para estudar a previsão de adesão a campanhas de marketing (variável alvo `y` com rótulos yes/no).

A escolha recai sobre um conjunto realista e público, com 41.188 observações e **20 atributos** de naturezas distintas (demográficas, socioeconômicas e de contato), o que permite testar um pipeline completo: engenharia de features, tratamento de desbalanceamento, treino/validação e interpretação de resultados.

**Dataframe escolhido:** UCI Bank Marketing – “bank-additional-full.csv”.

### Objetivo do projeto

* **Problema**: prever, **antes da abordagem**, se um cliente irá aderir à campanha (`y=yes`).
* **Uso prático**: priorizar contatos com **maior probabilidade de conversão**, otimizando esforço de telemarketing e taxa de sucesso.
* **Critérios de sucesso**: entregar um modelo com **F1 da classe `yes`** competitivo (dada a minoria da classe), com **recall** suficiente para não perder oportunidades relevantes e **precisão** que mantenha o custo de falsos positivos sob controle.

### Por que esse dataset?

Apesar de ser um problema clássico, ele está longe de ser “toy”: combina **variáveis categóricas** (`job`, `education`, `marital`, `contact`, `month`, `day_of_week`, etc.) com **numéricas** (`age`, `campaign`, `pdays`, `emp.var.rate`, `euribor3m`, `nr.employed`, entre outras). Essa mistura exige **one-hot encoding** e **padronização**. Outro ponto relevante é o **desbalanceamento moderado** (classe positiva \~11–12%), cenário ideal para discutir **métricas além de accuracy** (F1, *precision/recall*, **AUC-PR**) e técnicas como **`class_weight`** e ajuste de **limiar de decisão**.

In [ ]:
import sys, sklearn, tensorflow as tf
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [ ]:
# 1) Baixar e carregar o dataset
!wget -q https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
!unzip -o -q bank-additional.zip

In [ ]:
df = pd.read_csv('/content/bank-additional/bank-additional-full.csv', sep=';')
print(df.shape)
df.head(3)

(41188, 21)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [ ]:
# 2) EDA rápida
print("\n-> Distribuição da variável alvo:")
print(df['y'].value_counts(normalize=True).rename({'no':'no (%)','yes':'yes (%)'}))

print("\n-> Tipos das colunas:")
print(df.dtypes.value_counts())


-> Distribuição da variável alvo:
y
no (%)     0.887346
yes (%)    0.112654
Name: proportion, dtype: float64

-> Tipos das colunas:
object     11
int64       5
float64     5
Name: count, dtype: int64


# Exploração inicial do dataset

## Variável alvo (`y`)

* **Distribuição observada**:

  * **no**: **88,73%**
  * **yes**: **11,27%**
* **Implicações**:

  * Há **desbalanceamento** (classe positiva minoritária).
  * **Accuracy** pode ficar artificialmente alto se o modelo favorecer a classe majoritária; por isso, é importante monitorar **F1** (principalmente da classe `yes`), **matriz de confusão** e, se possível, **AUC-PR**.
  * Técnicas recomendadas:

    * **`class_weight`** (ponderar a perda para penalizar mais os erros na classe `yes`);
    * **Ajuste de limiar** de decisão (ex.: testar *thresholds* entre 0,3–0,5 para maximizar F1);
    * (Opcional) **Reamostragem** (*oversampling* da minoria ou *undersampling* da maioria) caso queira comparar.

## Tipos de variáveis (features)

* **11 colunas categóricas** (`object`): requerem **codificação** (ex.: *one-hot encoding*).
* **5 colunas inteiras** (`int64`): normalmente representam contagens/ordinais; podem ser usadas direto, mas é bom verificar escalas e possíveis outliers.
* **5 colunas numéricas contínuas** (`float64`): recomendável **padronizar** ou **normalizar** (ex.: `StandardScaler`) para estabilizar o treino do modelo.

## Consequências para o pipeline

1. **Pré-processamento**

   * One-hot nas categóricas (evitar ordem artificial).
   * *Scaling* nas numéricas (inteiras/contínuas), principalmente se usar otimização baseada em gradiente.
2. **Validação**

   * **Divisão estratificada** (preserva a proporção de classes em treino/teste).
   * Acompanhar **F1** além de *accuracy*; reportar também **precision** e **recall** da classe `yes`.
3. **Treinamento**

   * Modelo sequencial com uma **Dense(sigmoid)**, otimizador **Adam** e perda **`binary_crossentropy`**.
   * Considerar **`class_weight`** durante o ajuste para mitigar o desbalanceamento.
4. **Interpretação dos resultados**

   * Avaliar a matriz de confusão para entender onde o modelo erra (falsos negativos vs. falsos positivos).
   * Ajustar o **limiar** visando o objetivo do negócio (ex.: priorizar recall se é pior perder um `yes`).

> Em resumo, o dataset é adequado para classificação binária com **complexidade real**: mistura de variáveis categóricas e numéricas e **desbalanceamento moderado** que exige cuidados na avaliação (F1) e no treinamento (class weights/threshold).

In [ ]:
# 3) Pré-processamento
X = pd.get_dummies(df.drop(columns=['y']), drop_first=True)   # one-hot
y = (df['y'] == 'yes').astype('int32').values                 # 1 = yes

X_train, X_test, y_train, y_test = train_test_split(
    X.values, y, test_size=0.20, stratify=y, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train).astype('float32')
X_test  = scaler.transform(X_test).astype('float32')

**Falando sobre o pré-processamento**

Acima, estamos pegando o dataframe original, tirando a coluna alvo `y` e transformando todas as variáveis categóricas em indicadores 0/1 com `get_dummies`. O `drop_first=True` deixa uma categoria de cada variável como referência para evitar redundância (a “dummy trap”). Em seguida, transformamos o alvo em binário: `yes` vira 1, `no` vira 0 — é isso que o modelo vai aprender a prever.

Depois separamos os dados em treino (80%) e teste (20%) com estratificação para manter a mesma proporção de `yes`/`no` nos dois conjuntos. O `random_state=42` só fixa a aleatoriedade para que o resultado seja repetível. Repare que usamos `X.values`: isso converte o `DataFrame` em um array NumPy (perde os nomes das colunas), o que é suficiente para treinar a rede.

Por fim, padronizamos as features com `StandardScaler`: ajustamos média e desvio **só no treino** (`fit_transform`) para não vazar informação, e aplicamos a mesma transformação no teste (`transform`). Isso coloca as variáveis numa escala parecida (média 0, desvio 1), ajudando o otimizador a convergir. Convertê-las para `float32` reduz memória e costuma acelerar o treino em Keras. Observação: esse fluxo também escala as dummies 0/1, o que não é errado para redes neurais; se preferir, dá para manter `X` como `DataFrame` até o `train_test_split` e aplicar o `StandardScaler` apenas nas colunas numéricas originais.

In [ ]:
# Aqui, iremos lidar com leve desbalanceamento com class_weight
cw = compute_class_weight(class_weight='balanced', classes=np.array([0,1]), y=y_train)
class_weight = {0: float(cw[0]), 1: float(cw[1])}
print("\nClass weights:", class_weight)


Class weights: {0: 0.5634790341336616, 1: 4.438308189655173}


In [ ]:
# 4) Modelo Keras: uma camada Dense (sigmoid)
tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

hist = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=10,
    validation_split=0.1,
    verbose=0,
    class_weight=class_weight
)

In [ ]:
# 5) Avaliação (accuracy + F1)
y_proba = model.predict(X_test, verbose=0).ravel()
y_pred = (y_proba >= 0.5).astype('int32')

print("\nAccuracy (teste):", round(accuracy_score(y_test, y_pred), 4))
print("F1 (teste):      ", round(f1_score(y_test, y_pred), 4))
print("\nRelatório completo:\n", classification_report(y_test, y_pred, target_names=['no','yes']))


Accuracy (teste): 0.8634
F1 (teste):       0.6006

Relatório completo:
               precision    recall  f1-score   support

          no       0.99      0.86      0.92      7310
         yes       0.45      0.91      0.60       928

    accuracy                           0.86      8238
   macro avg       0.72      0.88      0.76      8238
weighted avg       0.93      0.86      0.88      8238



### Resultados (teste)

* **Acurácia**: **0,8634**
* **Classe `yes`** (928 amostras): **Precision 0,45 · Recall 0,91 · F1 0,6006**
Detecta a maioria dos positivos, mas com ~1.030 falsos positivos (precision baixa).

* **Classe `no`** (7310 amostras): **Precision 0,99 · Recall 0,86 · F1 0,92**
* **Médias**: **Macro F1 0,76** · **Weighted F1 0,88**

Percebemos que o uso de `class_weight` priorizou recall de yes. Foi bom para não perder positivos.


### Conclusão

* O modelo **prioriza recall em `yes`** (quase todos os casos positivos são pegos), **à custa de precisão** (muitos falsos positivos).
* A **acurácia** é alta principalmente porque a classe `no` é majoritária; **F1 da classe `yes`** é a métrica mais relevante aqui.

### Possíveis melhorias

1. **Escolha de limiar (threshold)**

  Hoje usamos 0,5. Talvez ajustando para uma maior precisão (aceitando um menor recall) pode ser melhor para o custo do projeto.

  Também podemos otimizar por F1 ou por uma função de custo (FP vs. FN).

2. **Regularização:**
  Talvez adicionar um L2 na camada para reduzir overfiting e estabilizar o limiar.
